# Wildfire

In [29]:
import sys

# required 3.12 version
if [sys.version_info[i] for i in range(3)][:2] != [3, 12]: 
    raise Exception(f"Python 3.12 is required (Current is {[sys.version_info[i] for i in range(3)]})")

import os
import time
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import imutils
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [30]:
PROGRAM_NAME: str = "Wildfire"
CATEGORIES: list[str] = ["fire", "nofire"]
DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

Wildfire Images:
> Ibn Jafar, Anam; Islam, Al Mohimanul ; Binta Masud, Fatiha; Ullah, Jeath Rahmat; Ahmed, Md. Rayhan (2023), “FlameVision : A new dataset for wildfire classification and detection using aerial imagery ”, Mendeley Data, V4, doi: 10.17632/fgvscdjsmt.4

In [31]:

from pathlib import Path

__file__ = current_directory = os.getcwd()
BASE_DIR = Path(__file__).resolve().parent / PROGRAM_NAME
DATA_DIR = BASE_DIR / "data"
CLASSIFICATION_DIR = DATA_DIR / "Classification"
CLASSIFICATION_TRAIN_DIR = CLASSIFICATION_DIR / "train"
CLASSIFICATION_TEST_DIR = CLASSIFICATION_DIR / "test"
CLASSIFICATION_VALID_DIR = CLASSIFICATION_DIR / "valid"
DETECTION_DIR = DATA_DIR / "Detection"
DETECTION_TRAIN_DIR = DETECTION_DIR / "train"
DETECTION_TEST_DIR = DETECTION_DIR / "test"
DETECTION_VALID_DIR = DETECTION_DIR / "valid"

In [32]:
def short_path_name(path: Path) -> str:
    parts = path.parts
    return '.../' + '/'.join(parts[-4:]) if len(parts) > 4 else str(path)

In [33]:
def generate_dataframe(folder_path: Path) -> pd.DataFrame:
    print(f"[{PROGRAM_NAME}]: Generating new dataframe from '{short_path_name(folder_path)}'")
    dataframe = pd.DataFrame([], columns = ["Image", "Class"])
    
    FIRE_DIR = folder_path / "fire"
    NOFIRE_DIR = folder_path / "nofire"
    
    fire_images = [(img.name, "fire") for img in FIRE_DIR.iterdir() if img.is_file()]
    nofire_images = [(img.name, "nofire") for img in NOFIRE_DIR.iterdir() if img.is_file()]

    data = fire_images + nofire_images
    dataframe = pd.DataFrame(data, columns=["Image", "Class"])

    return dataframe

In [34]:
classification_train_dataframe = generate_dataframe(CLASSIFICATION_TRAIN_DIR)
classification_test_dataframe = generate_dataframe(CLASSIFICATION_TEST_DIR)
classification_valid_dataframe = generate_dataframe(CLASSIFICATION_VALID_DIR)

[Wildfire]: Generating new dataframe from '.../Wildfire/data/Classification/train'
[Wildfire]: Generating new dataframe from '.../Wildfire/data/Classification/test'
[Wildfire]: Generating new dataframe from '.../Wildfire/data/Classification/valid'


In [35]:
print(classification_train_dataframe)

                 Image   Class
0         fire (1).png    fire
1        fire (10).png    fire
2       fire (100).png    fire
3      fire (1000).png    fire
4      fire (1001).png    fire
...                ...     ...
6795  nofire (984).png  nofire
6796   nofire (99).png  nofire
6797  nofire (997).png  nofire
6798  nofire (998).png  nofire
6799  nofire (999).png  nofire

[6800 rows x 2 columns]
